<a href="https://colab.research.google.com/github/seanreed1111/colab-demos/blob/master/GRADIO_file_editor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -qqq gradio loguru pyyaml
!gradio --version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5

In [4]:
import gradio as gr
import os
import json
import yaml

def read_file(file_path):
    _, ext = os.path.splitext(file_path)
    with open(file_path, 'r') as f:
        content = f.read()

    if ext == ".json":
        try:
            json_content = json.loads(content)
            yaml_content = yaml.dump(json_content, default_flow_style=False)
            return yaml_content, True, "yaml"
        except json.JSONDecodeError:
            return "Invalid JSON file!", False, "yaml"

    return content, False, "text"

def convert_content(content, current_format):
    if current_format == "yaml":
        try:
            json_content = yaml.safe_load(content)
            return json.dumps(json_content, indent=4), "json"
        except yaml.YAMLError:
            return "Invalid YAML content!", "yaml"
    else:
        try:
            json_content = json.loads(content)
            return yaml.dump(json_content, default_flow_style=False), "yaml"
        except json.JSONDecodeError:
            return "Invalid JSON content!", "json"

def save_file(content, file_path, directory, is_json, current_format):
    if not os.path.isdir(directory):
        return "Invalid directory!"

    file_name = os.path.basename(file_path)
    save_path = os.path.join(directory, file_name)

    if is_json:
        if current_format == "yaml":
            try:
                json_content = yaml.safe_load(content)
                with open(save_path, 'w') as f:
                    json.dump(json_content, f, indent=4)
            except yaml.YAMLError:
                return "Invalid YAML content!"
        else:
            try:
                json_content = json.loads(content)
                with open(save_path, 'w') as f:
                    json.dump(json_content, f, indent=4)
            except json.JSONDecodeError:
                return "Invalid JSON content!"
    else:
        with open(save_path, 'w') as f:
            f.write(content)

    return f"File saved successfully at {save_path}!"

def main():
    with gr.Blocks() as app:
        gr.Markdown("## File Editor App with JSON/YAML Conversion")

        file_input = gr.File(label="Upload a file", type="filepath")
        directory_input = gr.Textbox(label="Directory to Save", placeholder="Enter directory path")
        text_area = gr.Code(label="File Content", language="yaml", lines=20)
        toggle_button = gr.Button("Toggle JSON/YAML")
        save_button = gr.Button("Save Changes")
        save_status = gr.Textbox(label="Status", interactive=False)
        is_json_state = gr.State()
        current_format_state = gr.State()

        def update_editor(file_path):
            content, is_json, current_format = read_file(file_path)
            return content, is_json, current_format

        file_input.change(update_editor, inputs=file_input, outputs=[text_area, is_json_state, current_format_state])
        toggle_button.click(convert_content, inputs=[text_area, current_format_state], outputs=[text_area, current_format_state])
        save_button.click(save_file, inputs=[text_area, file_input, directory_input, is_json_state, current_format_state], outputs=save_status)

    app.launch()

if __name__ == "__main__":
    main()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ff2085586036c92636.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


That's all! Go ahead and open that share link in a new tab. Check out our [getting started](https://gradio.app/getting_started.html) page for more complicated demos.

In [5]:
import gradio as gr
import os
import json
import yaml

def read_file(file_path):
    _, ext = os.path.splitext(file_path)
    with open(file_path, 'r') as f:
        content = f.read()

    if ext == ".json":
        try:
            json_content = json.loads(content)
            yaml_content = yaml.dump(json_content, default_flow_style=False)
            return content, yaml_content, True, "yaml"
        except json.JSONDecodeError:
            return content, "Invalid JSON file!", False, "yaml"

    return content, content, False, "text"

def convert_content(content, current_format):
    if current_format == "yaml":
        try:
            json_content = yaml.safe_load(content)
            return json.dumps(json_content, indent=4), "json"
        except yaml.YAMLError:
            return "Invalid YAML content!", "yaml"
    else:
        try:
            json_content = json.loads(content)
            return yaml.dump(json_content, default_flow_style=False), "yaml"
        except json.JSONDecodeError:
            return "Invalid JSON content!", "json"

def save_file(content, file_name, directory, is_json, current_format):
    if not os.path.isdir(directory):
        return "Invalid directory!"

    save_path = os.path.join(directory, file_name)

    if is_json:
        if current_format == "yaml":
            try:
                json_content = yaml.safe_load(content)
                with open(save_path, 'w') as f:
                    json.dump(json_content, f, indent=4)
            except yaml.YAMLError:
                return "Invalid YAML content!"
        else:
            try:
                json_content = json.loads(content)
                with open(save_path, 'w') as f:
                    json.dump(json_content, f, indent=4)
            except json.JSONDecodeError:
                return "Invalid JSON content!"
    else:
        with open(save_path, 'w') as f:
            f.write(content)

    return f"File saved successfully at {save_path}!"

def main():
    with gr.Blocks() as app:
        gr.Markdown("## File Editor App with JSON/YAML Conversion and File Naming")

        file_input = gr.File(label="Upload a file", type="filepath")
        directory_input = gr.Textbox(label="Directory to Save", placeholder="Enter directory path")
        file_name_input = gr.Textbox(label="New File Name", placeholder="Enter new file name")
        original_text = gr.Code(label="Original File Content", language="yaml", lines=20, interactive=False)
        editable_text = gr.Code(label="Editable Content", language="yaml", lines=20)
        toggle_button = gr.Button("Toggle JSON/YAML")
        save_button = gr.Button("Save Changes")
        save_status = gr.Textbox(label="Status", interactive=False)
        is_json_state = gr.State()
        current_format_state = gr.State()

        def update_editor(file_path):
            original, content, is_json, current_format = read_file(file_path)
            return original, content, is_json, current_format, os.path.basename(file_path)

        file_input.change(update_editor, inputs=file_input, outputs=[original_text, editable_text, is_json_state, current_format_state, file_name_input])
        toggle_button.click(convert_content, inputs=[editable_text, current_format_state], outputs=[editable_text, current_format_state])
        save_button.click(save_file, inputs=[editable_text, file_name_input, directory_input, is_json_state, current_format_state], outputs=save_status)

    app.launch()

if __name__ == "__main__":
    main()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://112558364c57c3fa3b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [6]:
import gradio as gr
import os
import json
import yaml

def read_file(file_path):
    _, ext = os.path.splitext(file_path)
    with open(file_path, 'r') as f:
        content = f.read()

    if ext == ".json":
        try:
            json_content = json.loads(content)
            yaml_content = yaml.dump(json_content, default_flow_style=False)
            return content, yaml_content, True, "yaml"
        except json.JSONDecodeError:
            return content, "Invalid JSON file!", False, "yaml"

    return content, content, False, "text"

def convert_content(content, current_format):
    if current_format == "yaml":
        try:
            json_content = yaml.safe_load(content)
            return json.dumps(json_content, indent=4), "json"
        except yaml.YAMLError:
            return "Invalid YAML content!", "yaml"
    else:
        try:
            json_content = json.loads(content)
            return yaml.dump(json_content, default_flow_style=False), "yaml"
        except json.JSONDecodeError:
            return "Invalid JSON content!", "json"

def save_file(content, file_name, directory, is_json, current_format):
    if not os.path.isdir(directory):
        return "Invalid directory!"

    save_path = os.path.join(directory, file_name)

    if is_json:
        if current_format == "yaml":
            try:
                json_content = yaml.safe_load(content)
                with open(save_path, 'w') as f:
                    json.dump(json_content, f, indent=4)
            except yaml.YAMLError:
                return "Invalid YAML content!"
        else:
            try:
                json_content = json.loads(content)
                with open(save_path, 'w') as f:
                    json.dump(json_content, f, indent=4)
            except json.JSONDecodeError:
                return "Invalid JSON content!"
    else:
        with open(save_path, 'w') as f:
            f.write(content)

    return f"File saved successfully at {save_path}!"

def main():
    with gr.Blocks() as app:
        gr.Markdown("## File Editor App with JSON/YAML Conversion and File Naming")

        file_input = gr.File(label="Upload a file", type="filepath")
        directory_input = gr.Textbox(label="Directory to Save", placeholder="Enter directory path")
        file_name_input = gr.Textbox(label="New File Name", placeholder="Enter new file name")

        with gr.Column():
            original_text = gr.Code(label="Original File Content", language="yaml", lines=20, interactive=False)
            editable_text = gr.Code(label="Editable Content", language="yaml", lines=20)

        toggle_button = gr.Button("Toggle JSON/YAML")
        save_button = gr.Button("Save Changes")
        save_status = gr.Textbox(label="Status", interactive=False)
        is_json_state = gr.State()
        current_format_state = gr.State()

        def update_editor(file_path):
            original, content, is_json, current_format = read_file(file_path)
            return original, content, is_json, current_format, os.path.basename(file_path)

        file_input.change(update_editor, inputs=file_input, outputs=[original_text, editable_text, is_json_state, current_format_state, file_name_input])
        toggle_button.click(convert_content, inputs=[editable_text, current_format_state], outputs=[editable_text, current_format_state])
        save_button.click(save_file, inputs=[editable_text, file_name_input, directory_input, is_json_state, current_format_state], outputs=save_status)

    app.launch()

if __name__ == "__main__":
    main()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8ce08809b120848b60.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
import gradio as gr
import os
import json
import yaml

def read_file(file_path):
    _, ext = os.path.splitext(file_path)
    with open(file_path, 'r') as f:
        content = f.read()

    if ext == ".json":
        try:
            json_content = json.loads(content)
            yaml_content = yaml.dump(json_content, default_flow_style=False)
            return content, yaml_content, True, "yaml"
        except json.JSONDecodeError:
            return content, "Invalid JSON file!", False, "yaml"

    return content, content, False, "text"

def convert_content(content, current_format):
    if current_format == "yaml":
        try:
            json_content = yaml.safe_load(content)
            return json.dumps(json_content, indent=4), "json"
        except yaml.YAMLError:
            return "Invalid YAML content!", "yaml"
    else:
        try:
            json_content = json.loads(content)
            return yaml.dump(json_content, default_flow_style=False), "yaml"
        except json.JSONDecodeError:
            return "Invalid JSON content!", "json"

def save_file(content, file_name, directory, is_json, current_format):
    if not os.path.isdir(directory):
        return "Invalid directory!"

    save_path = os.path.join(directory, file_name)

    if is_json:
        if current_format == "yaml":
            try:
                json_content = yaml.safe_load(content)
                with open(save_path, 'w') as f:
                    json.dump(json_content, f, indent=4)
            except yaml.YAMLError:
                return "Invalid YAML content!"
        else:
            try:
                json_content = json.loads(content)
                with open(save_path, 'w') as f:
                    json.dump(json_content, f, indent=4)
            except json.JSONDecodeError:
                return "Invalid JSON content!"
    else:
        with open(save_path, 'w') as f:
            f.write(content)

    return f"File saved successfully at {save_path}!"

def main():
    with gr.Blocks() as app:
        # Add CSS for styling
        app.css = """
        #original, #editable {
            width: 350px; /* Adjust width as needed */
        }
        """

        gr.Markdown("## File Editor App with JSON/YAML Conversion and File Naming")

        file_input = gr.File(label="Upload a file", type="filepath")
        directory_input = gr.Textbox(label="Directory to Save", placeholder="Enter directory path")
        file_name_input = gr.Textbox(label="New File Name", placeholder="Enter new file name")

        with gr.Row():
            original_text = gr.Code(label="Original File Content", language="yaml", lines=20, elem_id="original")
            editable_text = gr.Code(label="Editable Content", language="yaml", lines=20, elem_id="editable")

        toggle_button = gr.Button("Toggle JSON/YAML")
        save_button = gr.Button("Save Changes")
        save_status = gr.Textbox(label="Status", interactive=False)
        is_json_state = gr.State()
        current_format_state = gr.State()

        def update_editor(file_path):
            original, content, is_json, current_format = read_file(file_path)
            return original, content, is_json, current_format, os.path.basename(file_path)

        file_input.change(update_editor, inputs=file_input, outputs=[original_text, editable_text, is_json_state, current_format_state, file_name_input])
        toggle_button.click(convert_content, inputs=[editable_text, current_format_state], outputs=[editable_text, current_format_state])
        save_button.click(save_file, inputs=[editable_text, file_name_input, directory_input, is_json_state, current_format_state], outputs=save_status)

    app.launch()

if __name__ == "__main__":
    main()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9ad664319754bb1a94.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
